# Racial bias analysis in police stops.

We want to study the data used by stanford in their paper to check if the characteristics of a police officer are linked to a potential racial bias in stops.

To this end, we will use the stops data of the state police of Florida, and analyse behaviors towards white, black, or hispanic people.

# Summary:
- [Data exploration](#Exploration)
- [Statewide bias score](#Statewide_score)
- [County adjusted bias score](#County_score)
- [Veil of darkness further study](#Veil_of_darkness)

## First, let's gather and clean the data:

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from copy import copy

from tqdm import tqdm
tqdm.pandas()

import statsmodels.api as sm
import statsmodels.formula.api as smf

C:\Users\lenovo\Anaconda3\envs\ada\lib\site-packages\tqdm\std.py:699: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [2]:
folder = 'data/'
state = folder + 'fl_statewide.csv.zip'

keep_columns = ['date', 'time', 'county_name', 'subject_age', 'subject_race', 'subject_sex', 'officer_id_hash', 'officer_age', 'officer_race', 'officer_sex', 'officer_years_of_service', 'arrest_made', 'citation_issued', 'warning_issued', 'frisk_performed', 'search_conducted']
mandatory_columns = ['date', 'time', 'subject_age', 'subject_race', 'subject_sex', 'officer_id_hash', 'officer_age', 'officer_race', 'officer_sex', 'officer_years_of_service', 'arrest_made', 'citation_issued', 'warning_issued', 'search_conducted']
minorities = ['white', 'hispanic', 'black']
boolean_columns = ['arrest_made', 'citation_issued', 'warning_issued', 'frisk_performed', 'search_conducted']

### Helper functions

In [3]:
def type_booleans(df):
    """ type a column in boolean if possible to reduce size and handability of dataframe
    """
    global boolean_columns
    for col in boolean_columns:
        if df_full[col].isna().sum() == 0:
            df_full[col] = df_full[col].astype(bool)
        else:
            print(f"Cannot convert {col} columns to boolean")

def print_info_df(df):
    """ print summary of dataframe and values in columns if not too long
    """
    print("Dataset is composed of {} stops. Columns are: \n".format(df.shape[0]))
    for col in df.columns:
        if df.dtypes[col] != np.float64:
            val = df[col].unique()
            if len(val) > 20:
                print('{} \t\t: too much different values'.format(col))
            else:
                print('{} \t\t: values are: {}'.format(col if len(col)>15 else col + "\t\t", val))

def generate_smaller_data(df, keep_ratio, path):
    """ Write new csv of reduced size in path and returns it
    """
    total_size = df.shape[0]
    df_red = df.sample(n=int(total_size * keep_ratio))
    df_red.to_csv(path, index = False)
    return df_red

## Loading and cleaning florida dataset

In [ ]:
# load all dataset
df_full = pd.read_csv(state)
print(len(df_full))

In [ ]:
df_full.drop(columns=df_full.columns.difference(keep_columns), inplace=True) # drop unused columns
df_full.dropna(subset=mandatory_columns, how='any', inplace=True) # drop nan values in mandatory columns
df_full['date'] = pd.to_datetime(df_full['date']) # to datetime
df_full['year'] = df_full['date'].dt.to_period('y')
df_full = df_full.rename(columns={'officer_years_of_service': 'officer_yos'})

print(len(df_full))
df_full.head()

<a id = Exploration></a>
## Data exploration

In [ ]:
# grasp on the data : is there spelling mistakes for gender, race; nan values; ...
print_info_df(df_full)

In [ ]:
# keep same minorities as papers 
# df_full = df_full[df_full['subject_race'].isin(minorities)]
df_full = df_full[df_full['officer_race'].isin(minorities)]
type_booleans(df_full)

<a id = Statewide_score></a>
# Bias score
## Goal
The goal here is to determined *how* the characteristics of an officer influence its bias towards minorities 

## How
- An officer : arrest over a N-year period
- Raw bias score for officer o towards minority m $ B(o,m) = \frac{N_{stops~of~m}}{N_{total~stops~over~the~period}} $
- Strong assumption: $median(\{B(o,m) / o \in S\})$ is actually the proportion of the population of $S$ which is from minority $m$ 
- Bias score for minority $m$ for an officer in a set of officers of region $S$ : $ B_m = \frac{N_{stops~of~m}}{N_{total~stops~over~the~period}} - median(\{B(o,m) / o \in S\})$

The study is held this way : 
- $S$ is statewide : see the coefficient statewide

## Definition of first officer dataframe

In [ ]:
df_officers = df_full.groupby(['year', 'officer_id_hash', 'officer_race', 'officer_sex'])['officer_yos', 'officer_age']
df_officers = df_officers.min().reset_index()
df_officers.set_index(['year', 'officer_id_hash'], inplace=True, verify_integrity=True)
df_officers.head()

In [ ]:
# build dataframe linking (year, officer) to their number of arrest of minorities
df_yearly = df_full.groupby(['year', 'officer_id_hash', 'subject_race'])['date'].count().to_frame().reset_index()
df_yearly.rename(columns={'date':'stops'}, inplace=True)
df_yearly = df_yearly.pivot_table(columns='subject_race', values='stops', index=['year', 'officer_id_hash'], fill_value=0)
df_yearly['total'] = df_yearly.sum(axis=1)
df_yearly.head()

In [ ]:
# drop officers with too few arrest
stop_threshold = 300
df_yearly = df_yearly[df_yearly['total'] > stop_threshold]
print(f'There are {len(df_yearly)} entries left')

In [ ]:
# add raw bias
for m in minorities:
        df_yearly[f'raw_bias_{m}'] = df_yearly[m] / df_yearly['total']

# compute medians
raw_bias_medians = { m : df_yearly[f'raw_bias_{m}'].median() for m in minorities}

# add bias
for m in minorities:
    df_yearly[f'bias_{m}'] = df_yearly[f'raw_bias_{m}'] - raw_bias_medians[m]

df_yearly.head()

In [ ]:
bias_df = df_yearly.merge(df_officers, how='left', left_index=True, right_index=True, validate='one_to_one')
bias_df.head(1)

## Now that we have the biases and the officer characteristics, let's look at it more closely:

In [ ]:
percents = ['bias_black', 'bias_white', 'bias_hispanic']
mapping = dict(zip(minorities, ['k'; 'r', 'y']))
medians = {}
for m in minorities:
    medians[color] = bias_df.loc[bias_df['officer_race'] == color, percents].median()

In [ ]:
plt.figure(figsize=(15,5))
for i, m in enumerate(minorities):
    plt.subplot(1,3,i+1)
    axs = sns.barplot(x = m, y = medians[m], color=mapping[m])
    axs.set_ylim(0, 0.7)
    plt.title(color + ' officer')
plt.show()

## Let's fit a linear regression to study a potential link between the officer features and the score.

In [ ]:
# fit models
for m in minorities:
    print()
    print(f'--------------{m.upper()}--------------')
    res = smf.ols(formula=f'bias_{m} ~ C(officer_race) + C(officer_sex) + officer_age', data=bias_df).fit()
    print(res.summary())

### Analysis
- The results are the opposite of what was expected : the bias score toward a minority m increases when the officer is of the same race
- The age seems to diminish the bias score of the officer

**Conclusion**
- The assumption "median of raw biases toward m = proportion of m in the local population" does not work.

In [ ]:
# number of counties in which an officer appears
county_per_officer = df_full.groupby('officer_id_hash')['county_name'].nunique()
sns.histplot(county_per_officer)

### Observations
All officers have more than 1 county.

<a id = County_score></a>
# County adjusted bias score.

## Goal

We previously attempted to define a statewide bias score to study the influence of officer's characteristics. <br/>
It is possible that differences in the the racial repartition of local population influence this score. In order to get rid of this effect, we tried to normalise the scores by county.

However, we noticed that officers often operate on multiple counties. Hence, it was not possible to just assign an officer to a county to adjust the scores. This is why the score will now be adjusted according to the county where *each stop* occures. 

In summary, each officer will have a "county score", and his bias score will be the mean of these scores, weighted by the proportion of stops he made in each counties.

The county score is the relative variation from the mean in the county.

For the county $c$ and minority $m$, we denote $x_{c, m} = \frac{\text{Number of stops of}~m~\text{in}~c}{\text{Total number of stops over the period in}~c}$

For officer $o$, we denote $p_{c, m, o} = \frac{\text{Number of stops of}~m~\text{in}~c~\text{by}~o}{\text{Total number of stops over the period in}~c~\text{by}~o}$


Finally, $N_o$ is the total number of stops by officer $o$, and $N_{o, c}$ is the number of stops by $o$ in county $c$.

We can now define an adjusted bias score for officer $o$ towards minority $m$ :

$$\boxed{\text{Score}_{o, m} = \frac{1}{N_o}\sum_c\left( N_{o,c} \cdot \frac{p_{c, m, o} - x_{c, m}}{x_{c, m}} \right)}$$

*N.B.* : minority denotes here only an ethnie, no matter the percentage of the population it represents.

### New helpers

In [ ]:
def create_year_hash(df):
    df['officer_hash_year'] = df['officer_id_hash'] + '-' + df['year'].astype(str)

#Used to compute the score of an officer
def score_by_county(officer_df, minority, county_means, relative = True):
    county_stop_proportion_minority = officer_df.groupby('county_name')[minority + '_stoped'].mean()
    county_stop_count = officer_df.groupby('county_name')[minority + '_stoped'].count()
    county_stop_prop = county_stop_count/county_stop_count.sum()
    if (relative is True):
        return (((county_stop_proportion_minority - county_means[minority].loc[county_stop_proportion_minority.index])/county_means[minority].loc[county_stop_proportion_minority.index])*county_stop_prop).sum()
    else:
        return ((county_stop_proportion_minority - county_means[minority].loc[county_stop_proportion_minority.index])*county_stop_prop).sum()
    
def age_map(x):
    if (x < 32):
        return "young"
    elif(x > 50):
        return "old"
    else:
        return "middle"

#### Creation of a hash by officer and year:

In [ ]:
df = df_full.copy()
create_year_hash(df)

In [ ]:
stops_required = 300

print("{:.0f}% of officers have enough stops.".format((df.groupby('officer_hash_year')['date'].count() > stops_required).mean()*100))

In [ ]:
officers_to_keep = df.groupby('officer_hash_year')['year'].count().loc[df.groupby('officer_hash_year')['year'].count() > stops_required].index

df = df[df.officer_hash_year.isin(officers_to_keep)]

print(len(df))

In [ ]:
for minority in minorities:
    df[minority + '_stoped'] = (df['subject_race'] == minority)

#Computation the % of stops for each minority in each county
county_means = {}
for minority in minorities:
    county_means[minority] = df.groupby('county_name')[minority + '_stoped'].mean()

#### Computation of the bias scores

In [ ]:
scores = {}
for minority in tqdm(minorities):
    scores[minority] = df.groupby('officer_hash_year').apply(score_by_county, minority, county_means)

#### Let's link these scores to the officers.

In [ ]:
officer_numerics = ['officer_age', 'officer_years_of_service']
officer_cat = ['officer_race', 'officer_sex']

# Create a dataframe with the characteristics of officers.
officer_df = df.groupby('officer_hash_year')[officer_numerics].mean()

officer_df[officer_cat] = (df[['officer_hash_year'] + officer_cat].drop_duplicates()).set_index('officer_hash_year')

# Add the bias score
for minority in minorities:
    officer_df[minority + '_bias'] = scores[minority]

## Once again, let's perform regressions to study the link between officer characteristics and scores.

In [ ]:
for minority in minorities:
    print()
    print(f'--------------{minority.upper()}--------------')
    res = smf.ols(formula=f'{minority}_bias ~ C(officer_race) + C(officer_sex) + officer_age + officer_years_of_service', data=officer_df).fit()
    print(res.summary())
    print()
    print()

### There does not seem to be a statistically significant link between the parameters considered and the score defined. Let's look directly at the distribution of scores according to the parameters.

In [6]:
biases = ['white_bias', 'hispanic_bias', 'black_bias']

minorities = ['white', 'hispanic', 'black']
sexes = ['male', 'female']
ages = ['young', 'middle', 'old']

choice_params = {'minority':minorities, 'sex':sexes, 'age': ages}
choice_category = {'minority':'officer_race', 'sex': 'officer_sex', 'age': 'officer_generation'}


In [ ]:
generation = officer_df['officer_age'].apply(age_map)

officer_df['officer_generation'] = generation

In [ ]:
choice = 'age' #Possibilities : 'age', 'sex', 'minority'

for bias in biases:
    axs =  plt.axes()
    for m in choice_params[choice]:
        sns.kdeplot(officer_df[officer_df[choice_category[choice]] == m][bias], label = m + " officers", ax = axs)
    axs.axvline(x=0, ymin=0, ymax=100, linestyle='dashed', color = 'grey')
    axs.set_xlim([-1.5,2])
    plt.legend()
    plt.show()

In [ ]:
mapping = {'hispanic': 'y', 
          'black': 'k', 
          'white':'r'}
colors = [mapping[r] for r in officer_df['officer_race']]

In [ ]:
from mpl_toolkits.mplot3d import axes3d

%matplotlib notebook

fig = plt.figure()
ax = plt.axes(projection="3d")
ax.scatter(officer_df['white_bias'],officer_df['black_bias'], officer_df['hispanic_bias'] , color=colors)
ax.set_xlabel('white_bias')
ax.set_ylabel('black_bias')
ax.set_zlabel('hispanic_bias')
plt.show()

In [ ]:
%matlplotlib inline

In [ ]:
print(officer_df.corr().loc[officer_numerics, biases])

# From those visualisations, there is no clear link between any of the parameters we chose and the defined bias score. 

This can be interpreted 2 different ways:
- The first differences were due to repartitions of population, which is why there were more officers stopping drivers of their own race.
- There is indeed a direct link between the characteristics and a bias, and a more adjusted metric could show it.

## Use of another metric:

The authors of the paper from which this work started used different kinds of metrics, in an atempt to produce objective results. We will try to use the veil of darkness that they introduced, with a light on the differences in this metric when the officer's characteristics are different.

In [ ]:
TODO : notebook Jonas

<a id = Veil_of_darkness></a>
# Veil of darness 